取得した文章に対して、想定される質問文の作成

In [16]:
from openai import OpenAI
import pandas as pd
import os

api_key = os.environ["OPENAI_API_KEY"]
# OpenAI クライアントの初期化
client = OpenAI(api_key=api_key)

# データフレームの取得
file_path = "C:/Users/iput Hata Sakuto/cuda-11.8/提出/ベースライン/out_put/database.xlsx" # <- 「use_df.xlsx」にて、カラムが空になっていたものを削除した
df = pd.read_excel(file_path)

In [17]:
df

,chunk,label,label_num
0,現代社会の課題は多様な要素が複雑に絡み合い、しかも常に変化しています。その解決には、単一の専...,専門職大学,0
1,だからこそ私たちは、既存の諸学問の壁を超えて専門分野を横断し、現場の実践で得られた知見や最新...,専門職大学,0
2,そのために、学生と教員が一体となって探究し、産業界や地域、世界の人々と協働する。理論と実践を...,専門職大学,0
3,そして、多様で複雑な社会課題に立ち向かう“Designer in Society（社会ととも...,専門職大学,0
4,これが、これからの時代に対応する新大学としての、私たち東京国際工科専門職大学の使命です。,専門職大学,0
5,55年ぶりに国がつくった新しい大学制度によって生まれた、職業に直結した教育を行う専門職大学。,専門職大学,0
6,プロフェッショナルになるために必要な知識と理論、そして実践的なスキルを身につけることができま...,専門職大学,0
7,東京国際工科専門職大学では、「プロフェッショナル」とは単に専門職に従事する人というだけではな...,専門職大学,0
8,学びが行き届く 「少人数教育」原則40人以下で行う少人数授業が、専門職大学の特長。本学ではさ...,特徴,1
9,実践を学べる「実務家教員」 教員の４割以上は企業等で実務を経験した「実務家教員」特に本学では...,特徴,1


## 質問生成

In [18]:
from langchain_openai import OpenAI
from langchain_community.embeddings import OpenAIEmbeddings
from langchain.chains import HypotheticalDocumentEmbedder
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

In [19]:
import openai

# dfからchunkデータを取得
base_chunks = df['chunk'].tolist()

# ChatGPTに要約をリクエスト
q_summarize_data = []
#print(len(base_chunks)*40)
total = len(base_chunks)*40
chunk_counter = 0

for i in range(total):
    su_target = base_chunks[chunk_counter]
    
    if i % 40 == 0:
        if i > 0:
            chunk_counter += 1
        
    response = client.chat.completions.create(
        model="gpt-3.5-turbo", 
        messages=[{"role": "user", "content": f"これに対応する質問文を100字以内で考えて: {su_target}"}]
    )
    
    # responseからcontentを抽出してsu_dataに格納
    content = response.choices[0].message.content
    q_summarize_data.append(content)

In [20]:
q_summarize_data

['現代社会において、多様な要素が複雑に絡み合い、常に変化している課題について、どのようなアプローチが必要ですか？',
 '現代社会の課題に対応するためには、どのような知識やスキルが必要でしょうか？',
 '現代社会の課題を解決するには、単一の専門分野の知識だけでは不十分ですか？',
 '現代社会の課題に対応するには、単一の専門知だけではなく、多様な要素を考慮し常に変化する知識＜動的な総合的知識＞が必要ではありませんか？',
 '現代社会の課題解決には、どのような知識やスキルが必要だと考えられますか？',
 '現代社会の課題への対応には、静的な専門知識だけでは不十分ですが、さまざまな要素を考慮しながら、どのように広範な知識やスキルを身につけるべきですか？',
 '現代社会の課題を解決するためには、静的な専門知では足りず、多様な要素が複雑に絡み合い、常に変化する状況に対応するための対策が必要ですか？',
 '現代社会の課題の多様性と変化に対応するために、どのようなスキルやアプローチが必要だと思いますか？',
 '現代社会の課題を解決するには、静的な専門知だけでは足りないとされていますが、それに対してはどのようなアプローチが考えられますか？',
 '現代社会の課題を解決するために必要な知識は、静的な専門知だけでは足りませんが、その具体的な要素や変化にどのように対応すべきですか？',
 '現代社会の課題を解決するために必要な要素や知識はどのように変化していますか？',
 '現代社会の課題解決には静的な専門知だけでは対応できない理由は何ですか？',
 '現代社会の課題の複雑さと変動性に対応するためには、どのようなスキルや知識が必要とされているのでしょうか？',
 '現代社会の課題を解決するためには、どのような知識や能力が必要ですか？',
 '現代社会の多様な問題を解決するためには、静的な専門知識だけでは不十分ですか？',
 '現代社会の課題を解決するには、どのようなアプローチやスキルが必要ですか？',
 '現代社会の課題を解決するためには、どのようなアプローチが必要ですか？',
 '現代社会の課題を解決するためには、静的な専門知だけでは不十分ですか？',
 '現代社会の課題を解決するために必要な要素は何ですか？',
 '現代社会の課題を解決するためには、静的な専門知だけでは

In [22]:
import pandas as pd

# summarize_dataがリストである場合
summarize_data_df = pd.DataFrame(q_summarize_data, columns=['chunk'])

summarize_data_df.to_excel('C:/Users/iput Hata Sakuto/cuda-11.8/提出/自作モデル-1/chunk2question.xlsx',index=False)

## 質問データのラベル設定

In [24]:
df = pd.read_excel('C:/Users/iput Hata Sakuto/cuda-11.8/提出/自作モデル-1/chunk2question.xlsx')
df

,Unnamed: 0,chunk
0,0,現代社会において、多様な要素が複雑に絡み合い、常に変化している課題について、どのようなアプロ...
1,1,現代社会の課題に対応するためには、どのような知識やスキルが必要でしょうか？
2,2,現代社会の課題を解決するには、単一の専門分野の知識だけでは不十分ですか？
3,3,現代社会の課題に対応するには、単一の専門知だけではなく、多様な要素を考慮し常に変化する知識＜...
4,4,現代社会の課題解決には、どのような知識やスキルが必要だと考えられますか？
...,...,...
1275,1275,深層学習のニューラルネットワークの技術内容と実現手法、効果について詳しく学び、具体的な適用方...
1276,1276,深層学習のニューラルネットワーク技術について、具体的な実装例を通じて学びますか？また、畳み込...
1277,1277,深層学習の主要技術であるニューラルネットワークの技術内容や実現手法、効果はどのようなものです...
1278,1278,深層学習の主要技術であるニューラルネットワークの技術内容と実現手法、効果について具体的な適用...


In [30]:
label_num_add = []
m = 0
num_add_counter = 0
total_size = len(df)
num_class = 4   #分類するクラス数
div_num = total_size // num_class #分割するターム

for m in range(len(df)):
  if m % div_num == 0:
    if m > 0:
      num_add_counter += 1
  label_num_add.append(num_add_counter)

print(label_num_add)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [34]:
321%4

1

In [36]:
label_num_add = []
m = 0
num_add_counter = 0
total_size = len(df)
num_class = 4   #分類するクラス数
div_num = total_size // num_class #分割するターム

for m in range(len(df)):
  if m % div_num == 1:
    print("No:",m)
    if m > 1:
      num_add_counter += 1
  label_num_add.append(num_add_counter)

print(label_num_add)

No: 1
No: 321
No: 641
No: 961
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 

In [37]:
df["label_num"] = label_num_add
df

,Unnamed: 0,chunk,label_num,label
0,0,現代社会において、多様な要素が複雑に絡み合い、常に変化している課題について、どのようなアプロ...,0,専門職大学
1,1,現代社会の課題に対応するためには、どのような知識やスキルが必要でしょうか？,0,専門職大学
2,2,現代社会の課題を解決するには、単一の専門分野の知識だけでは不十分ですか？,0,専門職大学
3,3,現代社会の課題に対応するには、単一の専門知だけではなく、多様な要素を考慮し常に変化する知識＜...,0,専門職大学
4,4,現代社会の課題解決には、どのような知識やスキルが必要だと考えられますか？,0,専門職大学
...,...,...,...,...
1275,1275,深層学習のニューラルネットワークの技術内容と実現手法、効果について詳しく学び、具体的な適用方...,3,授業
1276,1276,深層学習のニューラルネットワーク技術について、具体的な実装例を通じて学びますか？また、畳み込...,3,授業
1277,1277,深層学習の主要技術であるニューラルネットワークの技術内容や実現手法、効果はどのようなものです...,3,授業
1278,1278,深層学習の主要技術であるニューラルネットワークの技術内容と実現手法、効果について具体的な適用...,3,授業


In [38]:
df.loc[df["label_num"] == 0, "label"] = "専門職大学"
df.loc[df["label_num"] == 1, "label"] = "特徴"
df.loc[df["label_num"] == 2, "label"] = "学び"
df.loc[df["label_num"] == 3, "label"] = "授業"
df

,Unnamed: 0,chunk,label_num,label
0,0,現代社会において、多様な要素が複雑に絡み合い、常に変化している課題について、どのようなアプロ...,0,専門職大学
1,1,現代社会の課題に対応するためには、どのような知識やスキルが必要でしょうか？,0,専門職大学
2,2,現代社会の課題を解決するには、単一の専門分野の知識だけでは不十分ですか？,0,専門職大学
3,3,現代社会の課題に対応するには、単一の専門知だけではなく、多様な要素を考慮し常に変化する知識＜...,0,専門職大学
4,4,現代社会の課題解決には、どのような知識やスキルが必要だと考えられますか？,0,専門職大学
...,...,...,...,...
1275,1275,深層学習のニューラルネットワークの技術内容と実現手法、効果について詳しく学び、具体的な適用方...,3,授業
1276,1276,深層学習のニューラルネットワーク技術について、具体的な実装例を通じて学びますか？また、畳み込...,3,授業
1277,1277,深層学習の主要技術であるニューラルネットワークの技術内容や実現手法、効果はどのようなものです...,3,授業
1278,1278,深層学習の主要技術であるニューラルネットワークの技術内容と実現手法、効果について具体的な適用...,3,授業


In [40]:
df.to_excel('C:/Users/iput Hata Sakuto/cuda-11.8/提出/自作モデル-1/chunk2question_after.xlsx')